In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/cell_images/cell_images/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os.path,shutil
import gc
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing.image import load_img
from keras.utils import np_utils
import random
from sklearn.metrics import f1_score,confusion_matrix,precision_score,recall_score,accuracy_score
%matplotlib inline

In [ ]:
# create folder
os.makedirs("../working/train/Parasitized/")
os.makedirs('../working/validation/Parasitized/')
os.makedirs('../working/test/Parasitized/')
os.makedirs("../working/train/Uninfected/")
os.makedirs('../working/validation/Uninfected/')
os.makedirs('../working/test/Uninfected/')


In [ ]:
gc.collect()
# copy/split data into train/validation/testfolder

for path in os.listdir("../input/cell_images/cell_images/Parasitized/"):
    if path.split('.')[1]=='png':
        n=random.uniform(1,100)
        if n>=90:
            shutil.copyfile("../input/cell_images/cell_images/Parasitized/"+path,
                            "../working/test/Parasitized/"+path)
        elif ((80<=n) and (n<90)):
            shutil.copyfile("../input/cell_images/cell_images/Parasitized/"+path,
                            "../working/validation/Parasitized/"+path)
        elif n<80:
            shutil.copyfile("../input/cell_images/cell_images/Parasitized/"+path,
                            "../working/train/Parasitized/"+path)
for path in os.listdir("../input/cell_images/cell_images/Uninfected/"):
    if path.split('.')[1]=='png':
        n=random.uniform(1,100)
        if n>=90:
            shutil.copyfile("../input/cell_images/cell_images/Uninfected/"+path,
                            "../working/test/Uninfected/"+path)
        elif ((80<=n) and (n<90)):
            shutil.copyfile("../input/cell_images/cell_images/Uninfected/"+path,
                            "../working/validation/Uninfected/"+path)
        elif n<80:
            shutil.copyfile("../input/cell_images/cell_images/Uninfected/"+path,
                            "../working/train/Uninfected/"+path)

In [ ]:
# load test data for further use
test_data= []
test_label = []

        
for path in os.listdir("../working/test/Uninfected/"):
    if path.split('.')[1]=='png':
        img_read  = plt.imread("../working/test/Uninfected/"+path)
        image=cv2.resize(img_read,(64,64))
        image = img_to_array(image)
        test_data.append(image)
        test_label.append(0)
for path in os.listdir("../working/test/Parasitized/"):
    if path.split('.')[1]=='png':
        img_read  = plt.imread("../working/test/Parasitized/"+path)
        image=cv2.resize(img_read,(64,64))
        image = img_to_array(image)
        test_data.append(image)
        test_label.append(1)
test_data=np.array(test_data)
test_label = np.array(test_label)


In [ ]:
len(os.listdir("../working/train/Uninfected/")),len(os.listdir("../working/validation/Uninfected/")),len(os.listdir("../working/test/Uninfected/"))

In [ ]:
len(os.listdir("../working/train/Parasitized/")),len(os.listdir("../working/validation/Parasitized/")),len(os.listdir("../working/test/Parasitized/"))

In [ ]:
train_len = len(os.listdir("../working/train/Uninfected/"))+len(os.listdir("../working/train/Parasitized/"))
test_len = len(os.listdir("../working/test/Uninfected/"))+len(os.listdir("../working/test/Parasitized/"))
val_len = len(os.listdir("../working/validation/Uninfected/"))+len(os.listdir("../working/validation/Parasitized/"))

In [ ]:
#print out some images
fig=plt.figure(figsize=(10,40))
a=1
for i in range(20):
    fig.add_subplot(20,4,a)
    plt.imshow(plt.imread("../working/test/Parasitized/"+random.choice(os.listdir("../working/test/Parasitized/"))))
    a+=1

In [ ]:
#print out some images
fig=plt.figure(figsize=(10,40))
a=1
for i in range(20):
    fig.add_subplot(20,4,a)
    plt.imshow(plt.imread("../working/test/Uninfected/"+random.choice(os.listdir("../working/test/Uninfected/"))))
    a+=1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,precision_score,recall_score

In [ ]:
from keras.layers import BatchNormalization,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Softmax,Dropout,Input,SeparableConv2D
from keras.layers import DepthwiseConv2D,Dense,Activation,Flatten
from keras.layers import GlobalMaxPooling2D,Input

In [ ]:
from keras import optimizers,callbacks
from keras.models import Sequential
from tensorflow.python.keras import backend as K
from keras import backend as KB
from keras.preprocessing.image import ImageDataGenerator,load_img
size = 64
KB.clear_session()
model = Sequential()
model.add(SeparableConv2D(32, (3, 3),strides=(1,1), input_shape = (size, size, 3), activation = 'selu'))
model.add(SeparableConv2D(32, (3, 3),strides=(2,2), activation = 'selu'))
model.add(SeparableConv2D(32, (3, 3),strides=(1,1), activation = 'selu'))
model.add(SeparableConv2D(32, (3, 3),strides=(2,2), activation = 'selu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(2,activation = 'softmax'))
optimizer = optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
checkpointer = callbacks.ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)
model.compile(optimizer =optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())

In [ ]:
def train_grid(channel,train_len,add_layer=False,test_len,val_len):
    from keras import backend as KB
    
    score = {}
    for ch in channel:
        KB.clear_session()
        gc.collect()
        model = Sequential()
        model.add(SeparableConv2D(ch, (3, 3),strides=(1,1), input_shape = (size, size, 3), activation = 'selu'))
        model.add(SeparableConv2D(ch, (3, 3),strides=(2,2), activation = 'selu'))
        model.add(SeparableConv2D(ch, (3, 3),strides=(1,1), activation = 'selu'))
        model.add(SeparableConv2D(ch, (3, 3),strides=(2,2), activation = 'selu'))
        if add_layer:
            model.add(SeparableConv2D(ch, (3, 3),strides=(1,1), activation = 'selu'))
        model.add(GlobalAveragePooling2D())
        model.add(Dense(2,activation = 'softmax'))
        optimizer = optimizers.SGD(lr=lrate,decay=1e-6,momentum=0.9,nesterov=True)
        model.compile(optimizer =optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
        train_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255)
        test_gen=ImageDataGenerator(rescale=1./255)
        valid_gen=ImageDataGenerator(rescale=1./255)
        train_gen =train_gen.flow_from_directory("../working/train/",batch_size=batch,shuffle=True,target_size=(64,64),classes = ['Uninfected','Parasitized'])
        test_gen= test_gen.flow_from_directory("../working/test/",batch_size=test_len,shuffle=False,target_size=(64,64),classes = ['Uninfected','Parasitized'])
        valid_gen=valid_gen.flow_from_directory("../working/validation/",batch_size=batch,shuffle=True,target_size=(64,64),classes = ['Uninfected','Parasitized'])
        checkpointer = callbacks.ModelCheckpoint(filepath='/tmp/weights'+str(lrate)+str(batch)+'.hdf5', verbose=1, save_best_only=True)
        history = model.fit_generator(train_gen,
                              steps_per_epoch=train_len//batch,
                              epochs=45,
                              shuffle=False,
                              callbacks=[checkpointer,time_callback],validation_data=valid_gen,validation_steps=val_len//batch)
        model.load_weights('/tmp/weights'+str(channel)+'.hdf5')
        acc_t= model.evaluate_generator(test_gen,1)
        acc_v = model.evaluate_generator(valid_gen,val_len//batch)
        f1 = f1_score(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1))
        precision = precision_score(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1))
        recall = recall_score(test_label,np.argmax(model._generator(test_gen,1,verbose=1),axis=1))
        score[str(ch)] = {'history':history,'acc_t':acc_t,'acc_v':acc_v,'f1':f1,'precision':precision,'recall':recall}
    return score

score = train_grid([32,64],True,train_len,test_len,val_len)
          
  

In [ ]:

train_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255)
test_gen=ImageDataGenerator(rescale=1./255)
valid_gen=ImageDataGenerator(rescale=1./255)
train_gen =train_gen.flow_from_directory("../working/train/",batch_size=32,shuffle=True,target_size=(size,size),classes = ['Uninfected','Parasitized']
                                         )
test_gen= test_gen.flow_from_directory("../working/test/",batch_size=test_len,shuffle=False,target_size=(size,size),classes = ['Uninfected','Parasitized']
                                      )
valid_gen=valid_gen.flow_from_directory("../working/validation/",batch_size=32,shuffle=True,target_size=(size,size),classes = ['Uninfected','Parasitized']
                                       )


In [ ]:
print(train_gen.class_indices,valid_gen.class_indices,test_gen.class_indices)



In [ ]:
import time
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
time_callback=TimeHistory()

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=train_len//32,
                              epochs=50,
                              shuffle=False,
                              callbacks=[checkpointer,time_callback],validation_data=valid_gen,validation_steps=val_len//32)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['acc','val_acc'])
plt.xlabel('epoch')
plt.ylabel('accuracy')

In [ ]:
times = time_callback.times
times

In [ ]:

model.load_weights('/tmp/weights.hdf5')

print(model.evaluate_generator(valid_gen,val_len//32,verbose=1))

In [ ]:
print("acc:",model.evaluate_generator(test_gen,1))
print("f1:",f1_score(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1)))
print(confusion_matrix(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1)))
print("precision:",precision_score(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1)))
print("recall:",recall_score(test_label,np.argmax(model.predict_generator(test_gen,1,verbose=1),axis=1)))

In [ ]:
def gradcam(model,img_path,channel,size):
    x = plt.imread(img_path)
    x = cv2.resize(x,(size,size))
    x = img_to_array(x) 
    x =np.expand_dims(x,axis=0)
    pred = model.predict(x)
    class_idx = np.argmax(pred[0])
    class_output = model.output[:,class_idx]
    last_conv = model.get_layer('separable_conv2d_4')
    grads = KB.gradients(class_output,last_conv.output)[0]
    pooled_grad=KB.mean(grads,axis=(0,1,2))
    iterate = KB.function([model.input],[pooled_grad,last_conv.output[0]])
    pool_grad_val,conv_val = iterate([x])
    for i in range(channel):
        conv_val [:,:,i]*=pool_grad_val[i]
        
    heatmap = np.sum(conv_val,axis=-1)
    heatmap = np.maximum(heatmap,0)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255/np.max(heatmap)* heatmap) #normalize to 1-255 
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap,cv2.COLOR_BGR2RGB)
    return class_idx,heatmap
     

In [ ]:
fig = plt.figure(figsize=(15,100))
a=1
while a<30:
    path = random.choice(os.listdir("../working/test/Parasitized/"))
    img_path="../working/test/Parasitized/"+path
    class_pred,heatmap=gradcam(model,img_path,32,64)
    fig.add_subplot(20,3,a)
    plt.title('predict_class:'+('Uninfected' if class_pred==0 else 'Parasitized'))
    img = plt.imread(img_path)
    plt.imshow(img)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('heatmap')
    plt.imshow(heatmap)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('imposed')
    plt.imshow(cv2.addWeighted(img,0.7,np.array(heatmap*1./255,dtype='f'),0.3,0,))
    a+=1
        
    

In [ ]:
fig = plt.figure(figsize=(15,100))
a=1
while a<30:
    path = random.choice(os.listdir("../working/test/Uninfected/"))
    img_path="../working/test/Uninfected/"+path
    class_pred,heatmap=gradcam(model,img_path,32,64)
    fig.add_subplot(20,3,a)
    plt.title('predict_class:'+('Uninfected' if class_pred==0 else 'Uninfected'))
    img = plt.imread(img_path)
    plt.imshow(img)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('heatmap')
    plt.imshow(heatmap)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('imposed')
    plt.imshow(cv2.addWeighted(img,0.7,np.array(heatmap*1./255,dtype='f'),0.3,0,))
    a+=1

In [ ]:
np.argmax(model.predict(test_data),axis=1)

In [ ]:

ng_idx=np.where(np.argmax(model.predict(test_data),axis=1)!=test_label)
ng_idx

In [ ]:
ng=test_data[ng_idx]
ng_label = test_label[ng_idx]
print(model.predict(np.expand_dims(ng[0],0)))
plt.imshow(ng[0])

In [ ]:
def gradcam_array(model,array,channel):
    array1=np.expand_dims(array,0)
    pred = model.predict(array1)
    class_idx = np.argmax(pred[0])
    print("predict_class_idx:",class_idx)
    class_output = model.output[:,class_idx]
    last_conv=model.get_layer("separable_conv2d_4")
    grads = KB.gradients(class_output,last_conv.output)[0]
    pooled_grad=KB.mean(grads,axis=(0,1,2))
    iterate = KB.function([model.input],[pooled_grad,last_conv.output[0]])
    pool_grad_val,conv_val = iterate([array1])
    for i in range(channel):
        conv_val [:,:,i]*=pool_grad_val[i]
        
    heatmap = np.sum(conv_val,axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap = cv2.resize(heatmap, (64, 64))
    heatmap = np.uint8((255/np.max(heatmap)) * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap,cv2.COLOR_BGR2RGB)
    return class_idx,heatmap
   

In [ ]:
fig = plt.figure(figsize=(15,100))
a=1
while a<30:
    arr,label = random.choice(list(zip(ng,ng_label)))
    class_pred,heatmap=gradcam_array(model,arr,32)
    fig.add_subplot(20,3,a)
    plt.title('predict_class:'+('Uninfected' if class_pred==0 else 'Parasitized')+"/actual:"+('Uninfected' if label==0 else 'Parasitized'))
    plt.imshow(arr)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('heatmap')
    plt.imshow(heatmap)
    a+=1
    fig.add_subplot(20,3,a)
    plt.title('imposed')
    plt.imshow(cv2.addWeighted(arr,0.7,np.array(heatmap*1./255,dtype='f'),0.3,0,))
    a+=1
        

from tensorflow.python.keras import layers as kl

class BatchNormalization(kl.BatchNormalization):
    def call(self, inputs, training=None):
        true_phase = int(K.get_session().run(K.learning_phase()))
        trainable = int(self.trainable)
        with K.learning_phase_scope(trainable * true_phase):
            ret = super(BatchNormalization, self).call(inputs, training)
        return ret

from keras import backend as KB
KB.clear_session()
gc.collect()
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as res_preprocess
from keras.models import Model
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
prediction = Dense(2,activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)
for layer in base_model.layers:
    layer.trainable = False

#optimizer = optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath='/tmp/resweights.hdf5', verbose=1, save_best_only=True,save_weights_only=True)
train_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,preprocessing_function=res_preprocess)
test_gen=ImageDataGenerator(preprocessing_function=res_preprocess)
valid_gen=ImageDataGenerator(preprocessing_function=res_preprocess)
train_gen =train_gen.flow_from_directory("../working/train/",batch_size=32,shuffle=True,target_size=(224,224),classes = ['Uninfected','Parasitized'])
test_gen= test_gen.flow_from_directory("../working/test/",batch_size=32,shuffle=False,target_size=(224,224),classes = ['Uninfected','Parasitized'])
valid_gen=valid_gen.flow_from_directory("../working/val/",batch_size=32,shuffle=True,target_size=(224,224),classes = ['Uninfected','Parasitized'])
print(train_gen.class_indices,valid_gen.class_indices,test_gen.class_indices)
model.summary()
history = model.fit_generator(train_gen,
                              steps_per_epoch=train//32,
                              epochs=15,
                              shuffle=False,
                              callbacks=[time_callback,checkpointer],validation_data=valid_gen,validation_steps=val//32)

#model.load_weights('/tmp/resweights.hdf5')
print(model.evaluate_generator(test_gen,test//32))

print("f1:",f1_score(test_label,np.argmax(model.predict_generator(test_gen,test//32),axis=1)))
print(confusion_matrix(test_label,np.argmax(model.predict_generator(test_gen,test//32),axis=1)))
print("precision:",precision_score(test_label,np.argmax(model.predict_generator(test_gen,test//32),axis=1)))
print("recall:",recall_score(test_label,np.argmax(model.predict_generator(test_gen,test//32),axis=1)))

KB.clear_session()
gc.collect()
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xce_preprocess
from keras.models import Model
base_model = Xception(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
prediction = Dense(2,activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)
for layer in base_model.layers:
    layer.trainable = False
#checkpointer = callbacks.ModelCheckpoint(filepath='/tmp/xceweights.hdf5', verbose=1, save_best_only=True,save_weights_only=True)
optimizer = optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(optimizer =optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy']) 
train_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,preprocessing_function=xce_preprocess)
test_gen=ImageDataGenerator(preprocessing_function=xce_preprocess)
valid_gen=ImageDataGenerator(preprocessing_function=xce_preprocess)
train_gen =train_gen.flow_from_directory("../working/train/",batch_size=32,shuffle=True,target_size=(299,299),classes = ['Uninfected','Parasitized'])
test_gen= test_gen.flow_from_directory("../working/test/",batch_size=32,shuffle=False,target_size=(299,299),classes = ['Uninfected','Parasitized'])
valid_gen=valid_gen.flow_from_directory("../working/val/",batch_size=32,shuffle=True,target_size=(299,299),classes = ['Uninfected','Parasitized'])
model.summary()
history = model.fit_generator(train_gen,
                              steps_per_epoch=train//32,
                              epochs=15,
                              shuffle=False,
                              callbacks=[time_callback],validation_data=valid_gen,validation_steps=val//32)    

#model.load_weights('/tmp/xceweights.hdf5')
print(model.evaluate_generator(test_gen,test//32))

print("f1:",f1_score(test_label,np.argmax(model.predict(xce_preprocess(test_data),axis=1))))
print(confusion_matrix(test_label,np.argmax(model.predict(xce_preprocess(test_data),axis=1))))
print("precision:",precision_score(test_label,np.argmax(model.predict(xce_preprocess(test_data),axis=1))))
print("recall:",recall_score(test_label,np.argmax(model.predict(xce_preprocess(test_data),axis=1))))

K.clear_session()
gc.collect()
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input as den_preprocess
base_model = DenseNet121(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
prediction = Dense(2,activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)
for layer in base_model.layers:
    layer.trainable = False
optimizer = optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
#checkpointer = callbacks.ModelCheckpoint(filepath='/tmp/denweights.hdf5', verbose=1, save_best_only=True,save_weights_only=True)
train_gen=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,preprocessing_function=den_preprocess)
test_gen=ImageDataGenerator(preprocessing_function=den_preprocess)
valid_gen=ImageDataGenerator(preprocessing_function=den_preprocess)
train_gen =train_gen.flow_from_directory("../working/train/",batch_size=32,shuffle=True,target_size=(224,224),classes = ['Uninfected','Parasitized'])
test_gen= test_gen.flow_from_directory("../working/test/",batch_size=32,shuffle=False,target_size=(224,224),classes = ['Uninfected','Parasitized'])
valid_gen=valid_gen.flow_from_directory("../working/val/",batch_size=32,shuffle=True,target_size=(224,224),classes = ['Uninfected','Parasitized'])
model.summary()
history = model.fit_generator(train_gen,
                              steps_per_epoch=train//32,
                              epochs=5,
                              shuffle=False,
                              callbacks=[time_callback],validation_data=valid_gen,validation_steps=val//32)

model.load_weights('/tmp/denweights.hdf5')
print(model.evaluate_generator(test_gen,test//32))

print("f1:",f1_score(test_label,np.argmax(model.predict(den_preprocess(test_data),axis=1))))
print(confusion_matrix(test_label,np.argmax(model.predict(den_preprocess(test_data),axis=1))))
print("precision:",precision_score(test_label,np.argmax(model.predict(den_preprocess(test_data),axis=1))))
print("recall:",recall_score(test_label,np.argmax(model.predict(den_preprocess(test_data),axis=1))))

KB.clear_session()
gc.collect()

In [ ]:
from fastai.vision import *
from fastai.core import *
from fastai import *

In [ ]:
tfms=get_transforms(flip_vert=True,max_zoom=1,max_warp=None,p_affine=0)
data = ImageDataBunch.from_folder("../working/",test='test',valid='validation',size=224,ds_tfms=tfms,classes =['Uninfected','Parasitized'])

In [ ]:
data.normalize()
data.show_batch(figsize=(10,10))

In [ ]:
score_board = {}
for m in [models.densenet121,models.resnet50,models.vgg16_bn]:
    learn = cnn_learner(data,m,metrics=accuracy)
    learn.fit_one_cycle(10)
    pred_v,y = learn.get_preds()
    pred_t = learn.get_preds(ds_type=DatasetType.Test)
    acc_t = accuracy_score(test_label,np.argmax(pred_t[0].numpy(),axis=1))
    acc_v = accuracy_score(y.numpy(),np.argmax(pred_v.numpy(),axis=1))
    f1 = f1_score(test_label,np.argmax(pred_t[0].numpy(),axis=1))
    precision = precision_score(test_label,np.argmax(pred_t[0].numpy(),axis=1))
    recall = recall_score(test_label,np.argmax(pred_t[0].numpy(),axis=1))
    score_board[m.__name__] = {'f1':f1,'precision':precision,'recall':recall,'acc_t':acc_t,'acc_v':acc_v}

In [ ]:
score_board

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(9,figsize=(15,15))